In [4]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment.
!ls /home/aistudio/data
!/Users/hexiangyu/Downloads/easyFL-main/venv/bin/python -m pip install --upgrade pip


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions.

In [ ]:
#!unzip data/data24982/VOC_MASK.zip -d masks

In [ ]:
#!unzip DATA/DATA1/JPEGImages.zip -d new_data/test/

In [ ]:
#!unzip data/data2019/dataset1.zip -d masks2

In [ ]:
import shutil
import os
in_path = 'new_data/test/JPEGImages'
out_path='masks/VOC_MASK/JPEGImages/'
for filepath,dirnames,filenames in os.walk(in_path):   #在多级目录下找文件
        for filename in filenames:
            str1 = filename.split('.')[0]
            str1_1 = filename.split('.')[1]
            if str1_1 == "jpg" or str1_1 == "jpeg" or str1_1 == "JPG" or str1_1 == "JPEG":
                shutil.copy(filepath + "/" + filename, out_path) #复制文件
                #shutil.move() 移动文件
            else:
                continue

In [ ]:
work_dir = 'masks2/dataset1/dataset1/Annotations'
new_dir='masks/VOC_MASK/Annotations/'
# masks/VOC_MASK/JPEGImages/1_Handshaking_Handshaking_1_341.jpg
for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
        for filename in filenames:
            file_path = os.path.join(parent, filename)
            file = open(file_path,"r+",encoding='UTF-8')
            newFile = open(new_dir+filename,"w",encoding='UTF-8')
            for line in file.readlines():
                if("FIRC" in line):
                    line = line.replace("FIRC","Unknown")
                if("person" in line):
                    line = line.replace("person","not_standard")
                newFile.writelines(line)
            print (filename)
            newFile.close()
            file.close()


In [ ]:
import xml.etree.cElementTree as ET
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import paddle.fluid as fluid
import myyolo3net as mynet
import time
import random



In [ ]:
xmlDir = r"masks/VOC_MASK/Annotations"
imgDir = r"masks/VOC_MASK/JPEGImages"
xmlList = [os.path.join(xmlDir, f) for f in os.listdir(xmlDir)]
labelDic = {'face': 0, 'face_mask': 1}
BATCH_SIZE = 16
imgSize = (448,448)  # w,h
iouLim = 0.1
# anchorBox = [(40, 160), (80, 210), (80, 400)]  # 前宽后高
anchorBox = [(40, 40), (120, 120), (220, 220)]  # 前宽后高
colNum = imgSize[1] // 32
rowNum = imgSize[0] // 32
anchorNum = len(anchorBox)
classNum = len(labelDic)
outDim = (5 + classNum) * anchorNum
learningRate = 1e-4
testSize=96
random.shuffle(xmlList)
xmlListTest=xmlList[-testSize:]
xmlList=xmlList[0:-testSize]


# 从xml读图片，标签
def loadXmlandImg(xmlDir, loadImg=False, imgDir=imgDir, resize=False):
    # 输出的size是一个元组（w，h），theObjs为列表，每一位是一个字典，键name->标签，键box->[xxyy]
    xml = ET.parse(xmlDir)
    size = (
        int(xml.find("size").find("width").text),
        int(xml.find("size").find("height").text)
    )

    objs = xml.findall("object")
    theObjs = []
    # 遍历每个物品框生成一堆字典
    for obj in objs:
        bndbox = obj.find("bndbox")
        boxXXYY = [int(bndbox.find("xmin").text), int(bndbox.find("xmax").text),
                   int(bndbox.find("ymin").text), int(bndbox.find("ymax").text)]
        if resize:
            boxXXYY[0] *= resize[0] / size[0]
            boxXXYY[1] *= resize[0] / size[0]
            boxXXYY[2] *= resize[1] / size[1]
            boxXXYY[3] *= resize[1] / size[1]
        theObjs.append({
            "label": labelDic[obj.find("name").text],
            "box": boxXXYY
        })
    # 选择是不是加载图片
    if loadImg:
        fileName = xml.find("filename").text
        fileName = os.path.join(imgDir, fileName)
        img = cv2.imread(fileName)
        if resize:
            img = cv2.resize(img, resize)
        img = (cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype("float32") / 255.0).transpose(2, 0, 1)
        return size, theObjs, img
    else:
        return size, theObjs




# 构造yolo3真实框，输入真实框和框数上限，输出一个（框数，4）的xywh数组
def yolo3GTBoxLabel(gtBoxes, resize,boxNum=50):
    # gtBoxes，一个字典，包含label:01234和box:[x,x,y,y]两个键值对
    # shape = d['shapes']

    thisLabel = [0 for i in range(boxNum)]
    gtBoxArr = [[0, 0, 0, 0] for i in range(boxNum)]

    # 遍历这个图的每一个真实框
    for i in range(min(len(gtBoxes),boxNum)):
        box = gtBoxes[i]
        points = box['box']
        x1 = points[0]
        x2 = points[1]
        y1 = points[2]
        y2 = points[3]
        label = box['label']
        gtx = (x1 + x2) / 2
        gty = (y1 + y2) / 2
        gtw = x2 - x1
        gth = y2 - y1
        gtBoxArr[i] = [gtx/resize[0], gty/resize[1], gtw/resize[0], gth/resize[1]]
        thisLabel[i] = label
    # print(gtBoxArr,thisLabel)
    return np.array(gtBoxArr, dtype="float32"), np.array(thisLabel, dtype="int32")


# s, o, i = loadXmlandImg(os.path.join(xmlDir, xmlList[0]), True, imgDir)
# print(i.shape)


# 定义画矩形框的程序
def draw_rectangle(bbox, currentAxis=None, edgecolor='k', facecolor='y', fill=False, linestyle='-'):
    # currentAxis，坐标轴，通过plt.gca()获取
    # bbox，边界框，包含四个数值的list， [x1, x2, y1, y2]
    # edgecolor，边框线条颜色
    # facecolor，填充颜色
    # fill, 是否填充
    # linestype，边框线型
    # patches.Rectangle需要传入左上角坐标、矩形区域的宽度、高度等参数
    if not currentAxis:
        currentAxis = plt.gca()
    rect = patches.Rectangle((bbox[0], bbox[2]), bbox[1] - bbox[0] + 1, bbox[3] - bbox[2] + 1, linewidth=1,
                             edgecolor=edgecolor, facecolor=facecolor, fill=fill, linestyle=linestyle)
    currentAxis.add_patch(rect)

def dataReader2(xmllist, img_size=imgSize):
    labels = []
    gtboxes = []
    imgs = []
    for i in xmllist:
        size, obj, img = loadXmlandImg(i, True, resize=img_size)
        imgs.append(img)
        # print(obj)
        box, label = yolo3GTBoxLabel(obj, resize=imgSize)
        labels.append(label)
        gtboxes.append(box)
    return np.array(gtboxes, dtype="float32"), np.array(labels, dtype="int32"), np.array(imgs, dtype="float32")


# 构建网络

maxGTbox = 50
inputImg = fluid.layers.data("image", shape=[None, 3, imgSize[1], imgSize[0]], dtype="float32")
# inputLabel = fluid.layers.data("label", shape=[None, anchorNum, 5 + classNum, rowNum, colNum], dtype="float32")
inputLabel = fluid.layers.data("label", shape=[None, maxGTbox], dtype="int32")
inputGtbox = fluid.layers.data("gtbox", shape=[None, maxGTbox, 4], dtype="float32")
(C0, C1, C2) = mynet.Darknet53(inputImg)  # 经过骨干网络提取特征
(r0, t0, P0) = mynet.yoloMain(C0, outDim)  # 得到yolo3的0级输出
r0_resize = fluid.layers.resize_nearest(input=r0, scale=2)
C1_New = fluid.layers.concat(input=[r0_resize, C1], axis=1)  # 和C1拼起来
(r1, t1, P1) = mynet.yoloMain(C1_New, outDim)  # yolo3的1级输出
r1_resize = fluid.layers.resize_nearest(input=r1, scale=2)
C1_New = fluid.layers.concat(input=[r1_resize, C2], axis=1)    # 和二级拼起来
(r2, t2, P2) = mynet.yoloMain(C1_New, outDim)
print(P0, P1, P2)

# lossAvg = mynet.yolo3LossNet(P0, inputLabel)
lossAll0 = fluid.layers.yolov3_loss(
    x=P0,
    gt_box=inputGtbox,
    gt_label=inputLabel,
    anchors=[7, 10, 12, 22, 24, 17,22, 45, 46, 33, 43, 88,85, 66, 115, 146, 275, 240],
    anchor_mask=[6,7,8],
    class_num=classNum,
    ignore_thresh=iouLim,
    downsample_ratio=32
)
lossAll1 = fluid.layers.yolov3_loss(
    x=P1,
    gt_box=inputGtbox,
    gt_label=inputLabel,
    anchors=[7, 10, 12, 22, 24, 17,22, 45, 46, 33, 43, 88,85, 66, 115, 146, 275, 240],
    anchor_mask=[3,4,5],
    class_num=classNum,
    ignore_thresh=iouLim,
    downsample_ratio=16
)
lossAll2 = fluid.layers.yolov3_loss(
    x=P2,
    gt_box=inputGtbox,
    gt_label=inputLabel,
    anchors=[7, 10, 12, 22, 24, 17,22, 45, 46, 33, 43, 88,85, 66, 115, 146, 275, 240],
    anchor_mask=[0, 1, 2],
    class_num=classNum,
    ignore_thresh=iouLim,
    downsample_ratio=8
)
lossAll = lossAll0 + lossAll1 + lossAll2
lossAvg = fluid.layers.mean(lossAll)
print(lossAvg)
testProgram = fluid.default_main_program().clone(for_test=True)
opt = fluid.optimizer.Momentum(
    learning_rate=fluid.layers.piecewise_decay(boundaries=[800,1500,4000], values=[1e-3,3e-4,1e-4,5e-5]),
    # learning_rate=1e-4,
    momentum=0.9,
    # regularization=fluid.regularizer.L2Decay(0.0005),
)
a=opt.minimize(lossAvg)
# 开始训练

In [ ]:
print("开始编译网络")
initPro = fluid.default_startup_program()
mainProg = fluid.default_main_program()

# place = fluid.CPUPlace()
place = fluid.CUDAPlace(0)
exe = fluid.Executor(place)
exe.run(program=initPro)

def testModel():
    miniBatch=4
    begin=0
    end=miniBatch
    lossAll=0.0
    while end<=testSize:
        b, l, ii = dataReader2(xmlListTest[begin:end])
        lo=exe.run(testProgram,
        feed={
                    'image': ii,
                    "label": l,
                    'gtbox': b
        },
        fetch_list=[lossAvg]
        )
        lossAll+=lo[0]
        print(lo[0])
        begin+=miniBatch
        end+=miniBatch
    print("损失：",lossAll/(testSize//miniBatch))



In [ ]:
!cd
fluid.io.load_persistables(exe,'./model', mainProg)

print("开始训练")
dataNum = len(xmlList)
for i in range(5):
    testModel()
    random.shuffle(xmlList)
    tic = time.time()
    begin = 0
    end = BATCH_SIZE
    while end <= dataNum:
        b, l, ii = dataReader2(xmlList[begin:end])
        # print(b.shape, l.shape, i.shape)
        for j in range(1):
            la = exe.run(
                program=mainProg,
                feed={
                    'image': ii,
                    "label": l,
                    'gtbox': b
                },
                fetch_list=[lossAvg]
            )
        # print(la)
        if begin%1024==0:
            print(i,begin,la, time.time() - tic)
        begin += BATCH_SIZE
        end += BATCH_SIZE

# b, l, i = dataReader2(xmlList[0:1])

# print(b.shape, l.shape, i.shape)

In [ ]:
# 保存模型
fluid.io.save_persistables(exe, './model', fluid.default_main_program())

In [ ]:
# 保存预测模型
fluid.io.save_inference_model("./refMaskModel/",
                              feeded_var_names=['image'],
                              target_vars=[P0,P1,P2],
                              executor=exe)

In [ ]:
# 打包下载
!zip -r refMaskModel6.zip refMaskModel

In [ ]:
# 预测模型
import cv2
import numpy as np
import paddle.fluid as fluid
import time
import matplotlib.pyplot as plt
import math
import matplotlib.patches as patches

print("开始运行")

fluid.Scope()

## 加载所有文件的路径，每个文件在数组中有一位
modelPath = r'refMaskModel'
dataMap = {'face': 0, 'face_mask': 1,'not_standard':2}
classNum = 3
limObj = 0.8
iouLim=0.1

## 加载模型
place = fluid.CUDAPlace(0)
mainProg = fluid.default_main_program()
exe = fluid.Executor(place)

[inference_program,  # 预测用的program
 feed_target_names,  # 一个str列表，它包含需要在预测 Program 中提供数据的变量的名称。
 fetch_targets] = fluid.io.load_inference_model(
    modelPath,
    exe
)


# 测试模型

def loadImg(path):
    img = cv2.imread(path)
    # print(img)
    img = cv2.resize(img, (448, 448), interpolation=cv2.INTER_AREA)
    img = (cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype("float32") / 255.0).transpose(2, 0, 1)
    return np.array([img], dtype="float32")


def sigma(x):
    return 1 / (1 + math.exp(-x))


def draw_rectangle(bbox, currentAxis=None, edgecolor='k', facecolor='y', fill=False, linestyle='-'):
    # currentAxis，坐标轴，通过plt.gca()获取
    # bbox，边界框，包含四个数值的list， [x1, x2, y1, y2]
    # edgecolor，边框线条颜色
    # facecolor，填充颜色
    # fill, 是否填充
    # linestype，边框线型
    # patches.Rectangle需要传入左上角坐标、矩形区域的宽度、高度等参数
    if not currentAxis:
        currentAxis = plt.gca()
    rect = patches.Rectangle((bbox[0], bbox[2]), bbox[1] - bbox[0] + 1, bbox[3] - bbox[2] + 1, linewidth=1,
                             edgecolor=edgecolor, facecolor=facecolor, fill=fill, linestyle=linestyle)
    currentAxis.add_patch(rect)


def getIOU(xxyy1, xxyy2):
    x2 = min(xxyy1[1], xxyy2[1])
    x1 = max(xxyy1[0], xxyy2[0])
    y1 = max(xxyy1[2], xxyy2[2])
    y2 = min(xxyy1[3], xxyy2[3])

    if (x2 < x1) or (y1 > y2):
        return 0.0
    else:
        i = (x2 - x1) * (y2 - y1)
        s1 = (xxyy1[1] - xxyy1[0]) * (xxyy1[3] - xxyy1[2])
        s2 = (xxyy2[1] - xxyy2[0]) * (xxyy2[3] - xxyy2[2])
        return i / (s1 + s2 - i)


def refImg(imgPath):
    # maybeBox = [[] for i in range(classNum)]
    maybeBox = []
    img = loadImg(imgPath)

    def logBox(pred, stride, archors):
        pred0_ = pred[0]
        archorNum = len(archors) // 2
        for i in range(pred0_.shape[1]):
            for j in range(pred0_.shape[2]):
                for p in range(archorNum):
                    if sigma(pred0_[4 + p * (5 + classNum), i, j]) > limObj:
                        res = pred0_[p * (5 + classNum):(p + 1) * (5 + classNum), i, j]
                        x = stride * (j + sigma(res[0]))
                        y = stride * (i + sigma(res[1]))
                        w = archors[p * 2] * math.exp(res[2])
                        h = archors[p * 2 + 1] * math.exp(res[3])
                        # max_ = (-1000)
                        # maxClass = -1
                        # 比较是脸还是口罩
                        if res[5] > res[6]:
                            maxClass = 0
                        else:
                            maxClass = 1

                        # for c in range(5, 5 + classNum):
                        #     if res[c] > max_:
                        #         max_ = res[c]
                        #         maxClass = c - 5

                        maybeBox.append({
                            "bbox": [x - w / 2, x + w / 2, y - h / 2, y + h / 2],
                            "obj": sigma(res[5 + maxClass]),
                            "state": 0,
                            "class": maxClass,
                            "stride": stride
                        })

    tic = time.time()
    pred0, pred1, pred2 = exe.run(
        program=inference_program,
        feed={feed_target_names[0]: img},
        fetch_list=fetch_targets
    )
    print(time.time() - tic)
    logBox(pred0, stride=32, archors=[85, 66, 115, 146, 275, 240])
    logBox(pred1, stride=16, archors=[22, 45, 46, 33, 43, 88])
    logBox(pred2, stride=8, archors=[7, 10, 12, 22, 24, 17])
    # print(maybeBox)
    plt.imshow(img[0].transpose(1, 2, 0))
    # goodBox0 = [[] for i in range(classNum)]
    # 去除多余的框
    # for i in range(classNum):
    while True:
        finish = True
        max_ = -1
        maxBox = 0
        for box in maybeBox:
            if box["state"] == 0 and box["obj"] > max_:
                finish = False
                max_ = box["obj"]
                maxBox = box
        if not finish:
            maxBox["state"] = 2
        for box in maybeBox:
            if box["state"] == 0 and getIOU(box["bbox"], maxBox["bbox"]) > 0.1:
                box["state"] = 1
        if finish:
            break

    # print("face")
    for box in maybeBox:
        if box["state"] == 2:
            print(box)
            if box["class"] == 0:
                draw_rectangle(box["bbox"], edgecolor='r')
            else:
                draw_rectangle(box["bbox"])

    # print("facemask")
    # for box in maybeBox[1]:
    #     if box["state"] == 2:
    #         print(box)
    #         draw_rectangle(box["bbox"])

    plt.show()


refImg(r"masks/VOC_MASK/JPEGImages/1_Handshaking_Handshaking_1_341.jpg")


In [ ]:
# for i in range(1,15):
#     refImg("mymask/%d.jpg"%i)